In [7]:
import numpy as np
import pandas as pd
import Bio
from Bio.Seq import Seq
import umap

#allow dataframe display to stretch across the screen
pd.set_option('display.max_colwidth', None)

Once seuqencing data was found to be much more highly variable than the input library of 3,200 sequences, I took two approaches to explore enrichment. The first approach uses peptide frequencies under the assumption that any mutation may generate more diversity that would be important at the peptide level, since the peptide is what is understood to interact with the free tryptophan within the ribosome exit tunnel.  

Later in the script, variants are mapped back to their closest original library member (via Levenshtein distance) in the event that the diversity just came from library prep/sequencing artifacts.

## Clean up inputs

In [8]:
# Define a function to normalize frequencies within the dataset to be a range from 0-1 for color mapping
def normalize_freq(f):
    freq_scaled = (f - freq_min) / (freq_max - freq_min)
    return round(freq_scaled,4)

In [9]:
# Define a function to translate the DNA sequence into protein sequence
def translate_dna_to_protein(dna_sequence):
    # Create a Biopython Seq object from the DNA sequence
    dna_seq = Seq(dna_sequence)
    # Use the translate() method to convert the DNA sequence into protein sequence
    protein_seq = dna_seq.translate()
    # Return the protein sequence as a string
    return str(protein_seq)


#### Reference

In [11]:
#import reference sequence and translate to protein
reference= pd.read_csv('./example_data/genscript_NGS_3.csv')
#reference['gs5trim']= reference['NGS'].str.split('AGGTACC').str[1].str[:63]
reference['Translation'] = reference['seq'].apply(translate_dna_to_protein).str.split('*').str[0]
#calculate the frequency of the reads within the dataset (these are )
reference['Frequency']=reference['reads']/reference['reads'].sum()
# Rescale the frequency values to between 0 and 1
freq_min, freq_max = min(reference['Frequency']), max(reference['Frequency'])
reference['Norm_freq'] = reference['Frequency'].apply(normalize_freq)
reference.rename({'seq':'Genotype'}, axis=1, inplace=True)
reference

,Genotype,reads,Translation,Frequency,Norm_freq
0,TATTATCAATATTTGATTTTTAAAATTGAAAATGGACAATTAAAAGAAATATTACCATGA,63,YYQYLIFKIENGQLKEILP,0.000031,0.0201
1,TTAAGTGTCAATATTTACTGGATTAAAACAAGAGAGATAAAAAGGAGCCCCACACCATGA,105,LSVNIYWIKTREIKRSPTP,0.000052,0.0335
2,ACGGGCGAGGGCGACAGCTTTGCACAAGCTTGCTTACTAAGATTGGCAAAAGCGCCATGA,1222,TGEGDSFAQACLLRLAKAP,0.000606,0.3903
3,GACGTAAAATCCAAAGGGTATCAGGTTCTTTGGCCGGCTGTTCAAAGGGATTTGCCATGA,492,DVKSKGYQVLWPAVQRDLP,0.000244,0.1571
4,GACGTGAAGTCCAAAGGGTATCAGGTTCTTTGGCCGGCTGTTCAAAGGGATTTGCCATGA,1001,DVKSKGYQVLWPAVQRDLP,0.000497,0.3197
...,...,...,...,...,...
3243,GCCGTGGTGGTGCTGGTGTTCGAAGGCTTCAAGCATTTGATGCGCCAACGGCGCCCTTAA,1277,AVVVLVFEGFKHLMRQRRP,0.000634,0.4079
3244,CTGTCGCAGTCGCGGCTGTACGCGCTTTTGAAAACCCACGAGTTGCACCGCTCCCCGTAG,1551,LSQSRLYALLKTHELHRSP,0.000770,0.4954
3245,AACATCAACAAAGCTCATTTCTTCGATCCAGAAACAGAACAAGTCATTAAATTACCGTAA,115,NINKAHFFDPETEQVIKLP,0.000057,0.0367
3246,AAACTGGATTTGAAGATGTATCAGAAAATTATTTCAGACAATTCACATCTCACTCCCTGA,298,KLDLKMYQKIISDNSHLTP,0.000148,0.0952


In [12]:
#generate count of translation products to generate statistics for peptides
ref_count=reference[['Translation','Norm_freq']]
ref_count.sort_values('Norm_freq', ascending=False).head()

,Translation,Norm_freq
1796,PVLLLVWLASRVAPVGQRP,1.0000
2019,DDLHQPFLGAAVRIASQLP,0.9796
1161,LLGAVGFVWDWRRSRELAP,0.9342
43,PDQGATFTVELPLRPVSQP,0.9304
2228,LQACQAYVKAISSRAPAPP,0.9000


The tnaC sequence is (MNIL)ICVTSKWFNIDNKIVDHRP and can be used as a reference. Below could be used as a way to track this sequence:

lib_count.loc[lib3_count['Translation'] == 'ICVTSKWFNIDNKIVDHRP'

#### Library 
Load library 1 and format for merging

In [13]:
#load library
lib1= pd.read_csv('./example_data/lib_1.csv')
#generate protein sequences from library
lib1= lib1[['Genotype']]
#copy the genotype to a sequence column so that it has standard format with other libs
lib1['Sequence'] = lib1['Genotype']
lib1['protein'] = lib1['Genotype'].apply(translate_dna_to_protein).str.split('*').str[0]
lib1.columns=['Sequence', 'Genotype', 'protein']

#use this option if you want to cut out anything small peptides for normalization purposes (likely cloning artifacts)
lib1= lib1.loc[lib1['protein'].str.len() > 18]

lib1

/Users/Mark/miniconda/envs/NumpyTutorial/lib/python3.10/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,Sequence,Genotype,protein
0,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,MKVLRPKFGTLLSLAYFLLP
1,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,MKVLRPKFGTLLSLAYFLLP
2,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,MKVLRPKFGTLLSLAYFLLP
3,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,MKVLRPKFGTLLSLAYFLLP
4,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGGTCTCCCTCGCCTATTTCCTCCTTCCTTAG,ATGAAGGTACTCCGACCCAAATTTGGAACCCTGGTCTCCCTCGCCTATTTCCTCCTTCCTTAG,MKVLRPKFGTLVSLAYFLLP
...,...,...,...
59524,CTTTGCGCGTCAGCAGTGGTTCCGAATCCGCAAAGCGCATTGGGCTATGACGTGCCCTGA,CTTTGCGCGTCAGCAGTGGTTCCGAATCCGCAAAGCGCATTGGGCTATGACGTGCCCTGA,LCASAVVPNPQSALGYDVP
59525,CTTTGCGCGTCAGCAGTGGTTCCGAATCCGCAAAGCGCATTGGGCTATGACGTGCCGTAA,CTTTGCGCGTCAGCAGTGGTTCCGAATCCGCAAAGCGCATTGGGCTATGACGTGCCGTAA,LCASAVVPNPQSALGYDVP
59526,GTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,GTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,VAGLQAAWGVYQAENGVTP
59527,GTGGCGGGATTGCAGGCTGCGTGGGGCTTCTATCAGGCCGAGAATGGCGTTACTCCCTAA,GTGGCGGGATTGCAGGCTGCGTGGGGCTTCTATCAGGCCGAGAATGGCGTTACTCCCTAA,VAGLQAAWGFYQAENGVTP


In [14]:
# Calculate relative proportion of membership for terminal peptides
# The terminal region of tnaC is responsible for stalling the ribosome, 
# so if cloning did not go as expected, the terminal region could still be 
# responsible for signal
lib1_count=lib1['protein'].str[-19:].value_counts(normalize=True).to_frame()

#Normalize the frequency of peptides in Lib1
freq_min, freq_max = min(lib1_count['protein']), max(lib1_count['protein'])
lib1_count['Norm_freq'] = lib1_count['protein'].apply(normalize_freq)

#label the library
lib1_count.reset_index(inplace=True)
lib1_count=lib1_count.drop('protein', axis=1)
lib1_count.columns=['Translation', 'Norm_freq1']

lib1_count


,Translation,Norm_freq1
0,GNNNNIYSLGSTVGTNHAP,1.0000
1,RYLLRDYRGGGHKSMLRNP,0.8681
2,IVSTSLYYRRRGAAPLLQP,0.8460
3,CIVINWWFYLRPGAEHHNP,0.3795
4,LAGLQAAWGVYQAENGVTP,0.2064
...,...,...
3882,YLLRDYRGGGHKSMLRKPL,0.0000
3883,RYVLRDYRGGGHKSMLRNP,0.0000
3884,RCLLRDYRGGGHKSMLRNP,0.0000
3885,RHSRRVFKCAYKALLPAAP,0.0000


#### Library 2
Load library 2 and format for merging

In [15]:
#load library
lib2= pd.read_csv('./example_data/lib_2.csv')
#drop extra columns
lib2.drop(['29','0'], axis = 1, inplace=True)
lib2.columns=['Sequence']
#split the constant region and take the terminal nucleotides from the stop codon (end of read)
lib2['Genotype']= lib2['Sequence'].str.split('GTACC').str[1].str[:74]
#remove empty records
lib2 = lib2[lib2['Genotype'].notna()]
#translate the variable region
lib2['protein'] = lib2['Genotype'].apply(translate_dna_to_protein).str.split('*').str[0]

#keep only fields that have values
lib2_filtered = lib2[lib2['protein'].astype(bool)]
#use this option if you want to cut out anything smaller than 20AA for normalization purposes
lib2_filtered= lib2_filtered.loc[lib2_filtered['protein'].str.len() > 18]

lib2_filtered

/Users/Mark/miniconda/envs/NumpyTutorial/lib/python3.10/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,Sequence,Genotype,protein
2,AGGTACCATGAAATCCTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,ATGAAATCCTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,MKSWRDCRLRGASIRPRMA
4,AGGTACCATGAAATCATGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCATGACTCCCTAA,ATGAAATCATGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCATGACTCCCTAA,MKSWRDCRLRGASIRPRMA
5,AGGTACCATGAATCTCCTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACCCCCTAA,ATGAATCTCCTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACCCCCTAA,MNLLAGLQAAWGVYQAENGVTP
6,AGGTACCATGAATATTCTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,ATGAATATTCTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,MNILAGLQAAWGVYQAENGVTP
7,AGGTACCATGAATAACCTGGCGGGAATGCAGGCTGCGAGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,ATGAATAACCTGGCGGGAATGCAGGCTGCGAGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,MNNLAGMQAARGVYQAENGVTP
...,...,...,...
48449,AGGTACCATGATTATCCTGACAGGTGGTGTTTTGTTTTGGGCGTTCAAAATAGGATTACTAAAAAAGAAACCCCCGTGATTTGCCCTACTGTAGCTGTCTCTTATACACATCTCCGAGCCCACGAGACCTAGCT,ATGATTATCCTGACAGGTGGTGTTTTGTTTTGGGCGTTCAAAATAGGATTACTAAAAAAGAAACCCCCGTGATT,MIILTGGVLFWAFKIGLLKKKPP
48450,AGGTACCATGAATAAACTAATAGTGAGTACAGCGCTGTATTATCGCAGGCGTGGTGCTGCACTGTTACTCTAGTCGTAATTTGACATTATGTAGCTGTCTCTTATACACATCTCCTAGCCCATGAGACCTAGCT,ATGAATAAACTAATAGTGAGTACAGCGCTGTATTATCGCAGGCGTGGTGCTGCACTGTTACTCTAGTCGTAATT,MNKLIVSTALYYRRRGAALLL
48451,AGGTACCATGAATAACCTGACAGGTGGTGTTTTGTTTTGGGCGTTCAAAATAGGATTACTAAAAAAGAAACCCTCGTGATTTGCACTTCTGTAGCTTTCTCTTATACACATCTCCGAGCCCACGAGACCTAGCT,ATGAATAACCTGACAGGTGGTGTTTTGTTTTGGGCGTTCAAAATAGGATTACTAAAAAAGAAACCCTCGTGATT,MNNLTGGVLFWAFKIGLLKKKPS
48454,AGGTACCATGAATATGCCCTTCTGTAGCTGTCTCTTATACACATCTCCGAGCCCACGAGACCTAGCTATCTCGTATGCCGTCTTCTGCTTGAAAAAAAATAGACTCCATCAGAGATCGGT,ATGAATATGCCCTTCTGTAGCTGTCTCTTATACACATCTCCGAGCCCACGAGACCTAGCTATCTCGTATGCCGT,MNMPFCSCLLYTSPSPRDLAISYA


In [16]:
#calculate relative proportion of membership of peptides
lib2_count=lib2_filtered['protein'].str[-19:].value_counts(normalize=True).to_frame()

#Normalize the frequency of peptides in Lib2
freq_min, freq_max = min(lib2_count['protein']), max(lib2_count['protein'])
lib2_count['Norm_freq'] = lib2_count['protein'].apply(normalize_freq)

#label the library
lib2_count.reset_index(inplace=True)
lib2_count=lib2_count.drop('protein', axis=1)
lib2_count.columns=['Translation', 'Norm_freq2']

lib2_count

,Translation,Norm_freq2
0,GNNNNIYSLGSTVGTNHAP,1.0000
1,RYLLRDYRGGGHKSMLRNP,0.7794
2,TGGVLFWAFKIGLLKKKPP,0.5588
3,RNSDGFFHNLIDSFKKKKS,0.5441
4,IVSTSLYYRRRGAAPLLQP,0.4118
...,...,...
219,MNTLVWFDFKSAWFAPPEP,0.0000
220,TGGVLFWAFIIGLLKKKPS,0.0000
221,LKVVVAFRLGDAARTLLPP,0.0000
222,CIVIIWWFYLRPGAEHHNP,0.0000


#### Library 3
Load library 3 and format for merging

In [17]:
#load library
lib3=pd.read_csv('./example_data/lib_3.csv')
#drop empty records
lib3=lib3[lib3['seq'].notna()]
#translate variant seqeunces to peptides
lib3['protein'] = lib3['seq'].apply(translate_dna_to_protein).str.split('*').str[0]
#put into standardized format of other libraries
lib3.columns=['Sequence', 'Genotype', 'protein']

#use this option if you want to cut out anything smaller than 20AA for normalization purposes
lib3= lib3.loc[lib3['protein'].str.len() > 18]

lib3

/Users/Mark/miniconda/envs/NumpyTutorial/lib/python3.10/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,Sequence,Genotype,protein
0,AGGTACC,CTGCGCTATCTACTGCGCGACTATCGCGGCGGCGGACACAAGTCCATGCTGCGCAATCCATGA,LRYLLRDYRGGGHKSMLRNP
1,AGGTACC,CTGCGCTATCTCCTGCGCGACTATCGCGGCGGCGGACACAAGTCCATGCTGCGCAATCCATGA,LRYLLRDYRGGGHKSMLRNP
4,AGGTACC,CTGGCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACTCCCTAA,LAGLQAAWGVYQAENGVTP
5,AGGTACC,CTGCGCTATCTCCTGCGCGACTATCGCGGCGGCGGACACAAGTCCATGCTGCGCAATCCATGA,LRYLLRDYRGGGHKSMLRNP
6,AGGTACC,CTGAAGGTACTCCGACCCAAATTTGGACCCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,LKVLRPKFGPLLSLAYFLLP
...,...,...,...
29799,AGGTACC,CTGGGCAACAACAACAATATATATTCCCTAGGCAGTACGGTTGGGACTAATCATGCACCTTAA,LGNNNNIYSLGSTVGTNHAP
29800,AGGTACC,CTGGACGCCGTCTTCACCGACTACCCGCGCCAAGCGAAGCAATGGAAAGAGAAAACGCCTTAA,LDAVFTDYPRQAKQWKEKTP
29801,AGGTACC,CTGCAGATGCCGGATGGGATCTTTGGCTGGAAGATGTCTGGCCGGGCGGTTGAGAAACCCTGA,LQMPDGIFGWKMSGRAVEKP
29802,AGGTACC,CTGATAGTGAGCACATCGCTGTATTACCGCAGGCGTGGCGCTGCACCGTTACTCAAGCCGTGA,LIVSTSLYYRRRGAAPLLKP


In [18]:
#calculate relative proportion of membership of peptides
lib3_count=lib3['protein'].str[-19:].value_counts(normalize=True).to_frame()
#Normalize the frequency of peptides in Lib3
freq_min, freq_max = min(lib3_count['protein']), max(lib3_count['protein'])
lib3_count['Norm_freq'] = lib3_count['protein'].apply(normalize_freq)

#label the library
lib3_count.reset_index(inplace=True)
lib3_count=lib3_count.drop('protein', axis=1)
lib3_count.columns=['Translation', 'Norm_freq3']

lib3_count

,Translation,Norm_freq3
0,RYLLRDYRGGGHKSMLRNP,1.0000
1,KVLRPKFGTLLSLAYFLLP,0.5032
2,IVSTSLYYRRRGAAPLLQP,0.4207
3,GNNNNIYSLGSTVGTNHAP,0.4135
4,RNSDGFFHNLIDSFKKKKS,0.1999
...,...,...
1998,QTKPQRFFGYSKPDMRNAP,0.0000
1999,IVSTLLYYCRRGAAPLLQP,0.0000
2000,EQLAAIWKNEAVVRLNKMP,0.0000
2001,DDYAKEFAGIIRFLLEKQL,0.0000


#### Combine libraries for mapping

In [19]:
# Join frequencies to reference
ref_1_df = pd.merge(ref_count, lib1_count, on='Translation', how='left')
ref_12_df = pd.merge(ref_1_df, lib2_count, on='Translation', how='left')
ref_123_df = pd.merge(ref_12_df, lib3_count, on='Translation', how='left').fillna(0)

# display dataframe
ref_123_df.sort_values('Norm_freq3')

,Translation,Norm_freq,Norm_freq1,Norm_freq2,Norm_freq3
0,YYQYLIFKIENGQLKEILP,0.0201,0.0000,0.0000,0.0000
2117,SADVWAWDCAELEWLQHKP,0.4395,0.0004,0.0000,0.0000
2118,KGELHLFNGRIQYFQPRKP,0.0489,0.0001,0.0000,0.0000
2119,LDALTSYKWVVLGDGHIRP,0.0814,0.0000,0.0000,0.0000
2120,PLKAQTYTGTIKWTLTAGP,0.0901,0.0000,0.0000,0.0000
...,...,...,...,...,...
1087,LDQWRDFDSWVFAHTEIQP,0.3223,0.0330,0.0294,0.1998
829,GNNNNIYSLGSTVGTNHAP,0.1147,1.0000,1.0000,0.4135
1592,IVSTSLYYRRRGAAPLLQP,0.3635,0.8460,0.4118,0.4207
2773,KVLRPKFGTLLSLAYFLLP,0.2712,0.1311,0.1912,0.5032


#### Load sequences to be used for embedding

In [20]:
sequences = list(ref_count['Translation'])
#sequences = list(lib1_count['Translation'])
#sequences = list(lib2_count['Translation'])
#sequences = list(lib3_count['Translation'])

sequences


['YYQYLIFKIENGQLKEILP',
 'LSVNIYWIKTREIKRSPTP',
 'TGEGDSFAQACLLRLAKAP',
 'DVKSKGYQVLWPAVQRDLP',
 'DVKSKGYQVLWPAVQRDLP',
 'DNSPSRFRIGPVTSPGTLP',
 'GILVNYWKTYILKPATYLP',
 'RRAIRLYFEYISEHREVEP',
 'LSLTWDFTKFGKSLQKQQP',
 'IKIFWSYSNQFESQRGYVP',
 'DFNDERFKRLIDFIKSLQP',
 'KLKITGFISADDLLKKIEP',
 'NFIETVWGAGYKFIGEVLP',
 'NYDKNFYATQLELIERIKP',
 'GQKELGFLYLTPIQKIINP',
 'GQKELGFLYLTPIQKIINP',
 'GQKELGFLYLTPIQKIINP',
 'INNVRLFDTQENIPNPLQP',
 'MGEGDSFAQACLLRLAKAP',
 'VNYELKYLLSKAEPKELTP',
 'LYLFDLFRISYFCHINLKP',
 'QANEKVFSIPELVNGGIEP',
 'FVIPGLYIYLTSKKSKLKP',
 'QEADEAYAVLSGLPLRLKP',
 'ELAGSTFANVFANKNEVQP',
 'LIITLPYTKKSRRKRKSCP',
 'HLNNLGWFDFRSTWIEPGP',
 'EIMKVIFVPDKLINLVVKP',
 'LRQNFQFFLAMTEKFIKIP',
 'CCIFSHYKSFRLAKEPLPP',
 'RNSDGFFHNLIDSFKKKKP',
 'SKFMPEWRQYEFLLEGRAP',
 'LLISMPYSKKTRRKRKSRP',
 'ERSLAIFAAGGARASFAQP',
 'PHEPLLYQEEFPHLRRHYP',
 'MGEGDGFAQACLLRLAKAP',
 'EDVVVTYQILVYGVLIPEP',
 'TSPTAYYTDLIQRILSPKP',
 'GKGKAKYDSNDNSIYNVAP',
 'EIGIENFIETRRGLGYIMP',
 'NGFVSRYSRAILSTGDGSP',
 'LGKQTSFGLGKIDI

In [21]:
#one hot encode peptide sequences, pad sequences with "-" if needed
amino_acids = 'ACDEFGHIKLMNPQRSTVWY-'
num_acids = len(amino_acids)

max_len = 20  # set the maximum length of the sequences
pad_char = "-"  # choose the padding character
pad_pos="left"

def one_hot_encode(seq):
    seq_len = len(seq)
    if seq_len > max_len:
        raise ValueError(f"Sequence {seq} is longer than the maximum length {max_len}")
    pad_len = max_len - seq_len
    if pad_len > 0:
        if pad_pos == "left":
            seq = pad_char * pad_len + seq
        else:
            seq = seq + pad_char * pad_len
    encoding = np.zeros((max_len, num_acids))
    for i, acid in enumerate(seq):
        if acid not in amino_acids:
            raise ValueError(f"Invalid amino acid {acid} in sequence {seq}")
        encoding[i, amino_acids.index(acid)] = 1
    return encoding
    
one_hot_sequences = [one_hot_encode(seq) for seq in sequences]

#flatten seqeunces for embedding
flattened_sequences = [seq.flatten() for seq in one_hot_sequences]


In [22]:
#embed sequences
umap_object = umap.UMAP()
umap_embeddings = umap_object.fit_transform(flattened_sequences)

#### Plot interative umaps

In [23]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, ColorBar, LassoSelectTool, DataTable, TableColumn, HoverTool, Button
from bokeh.transform import linear_cmap
from bokeh.palettes import Reds
from bokeh.layouts import column, row
from bokeh.io import curdoc
from umap import UMAP

### The script below is used to investigate individual libraries
The script is designed for stand alone outputs, and not for use in a python notebook; however, users can be helpful to explore the seqeunces of interest within this pipeline.  The dataframes of individual libraries and columns should be customized to the library of interest.

In order to use this in interactive mode, a user would have to save the dataframe of interest and put the script into a standalone python script and then call the script by: 'bokeh serve --show appname.py'

In [57]:
# Create a Bokeh ColumnDataSource with the embeddings and frequency values

source = ColumnDataSource(data=dict(
    x=umap_embeddings[:, 0],
    y=umap_embeddings[:, 1],
    Norm_freq=ref_123_df['Norm_freq1'],
    Translation=ref_123_df['Translation'],
))

hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("Translation", "@Translation"),
    ("Norm_freq1", "@Norm_freq1"),
])


# Create a linear color map that maps the frequency values to a red 8 point gradient, 
# but only use the top 7 in reverse so that dark red is highest and the lowest is just off white
color_mapper = linear_cmap(field_name='Norm_freq1', palette=Reds[8][6::-1], low=0, high=1)

# Create a scatter plot of the embeddings with the points colored by frequency
plot = figure(tools=['pan,wheel_zoom,box_zoom,reset',hover],title='UMAP Embeddings of Protein Sequences')
plot.scatter(x='x', y='y', source=source, color=color_mapper)
plot.add_tools(LassoSelectTool())

# Add a color bar to show the mapping between frequency and color
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0, 0))
plot.add_layout(color_bar, 'right')

# create a table to display the selected points
columns = [TableColumn(field=col, title=col) for col in ref_123_df.columns]
table = DataTable(source=source, columns=columns, width=400, height=400)


# create a button to save the selected points as a CSV file
button = Button(label="Save CSV", button_type="success")

# define a callback function for the button click
def save_csv():
    selected_indices = source.selected.indices
    if selected_indices:
        selected_data = {col: source.data[col][selected_indices] for col in source.column_names}
        selected_df = pd.DataFrame(selected_data)
        selected_df.to_csv("selected_points.csv", index=False)

button.on_click(save_csv)


# define a callback function for the selection change event
def selection_change_callback(attrname, old, new):
    selected_indices = source.selected.indices
    if selected_indices:
        selected_data = {col: source.data[col][selected_indices] for col in source.column_names}
        selected_source.data = selected_data
    else:
        selected_source.data = {col: [] for col in source.column_names}

# create a new data source for the selected points
selected_source = ColumnDataSource({col: [] for col in source.column_names})
selected_table = DataTable(source=selected_source, columns=columns, width=400, height=400)

# add a selection change callback to the data source
source.selected.on_change('indices', selection_change_callback)


# combine the plot, tables, and button into a layout
layout = column(row(plot, table, button))

# display the layout
show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



#### Below is to plot the full dataset against reference
Here, we save a csv of the data of interest so that the script can be run as standalone output, which will enable interactive selection and CSV generation from selected points.

In [30]:
umapdf= pd.DataFrame(data=dict(
    x=umap_embeddings[:, 0],
    y=umap_embeddings[:, 1],
    #Matched_Genotype=ref_123_df['Genotype'],
    Translation=ref_123_df['Translation'],
    Norm_freq=ref_123_df['Norm_freq'],
    Norm_freq1_corr=ref_123_df['Norm_freq1']-ref_123_df['Norm_freq'],
    Norm_freq2_corr=ref_123_df['Norm_freq2']-ref_123_df['Norm_freq'],
    Norm_freq3_corr=ref_123_df['Norm_freq3']-ref_123_df['Norm_freq'],
    Norm_freq1=ref_123_df['Norm_freq1'],
    Norm_freq2=ref_123_df['Norm_freq2'],
    Norm_freq3=ref_123_df['Norm_freq3']
    ))

umapdf.to_csv('./output/umap_df_pep.csv')



In [32]:
# Create a Bokeh ColumnDataSource with the embeddings and frequency values
source = ColumnDataSource(data=dict(
    x=umap_embeddings[:, 0],
    y=umap_embeddings[:, 1],
    Norm_freq=ref_123_df['Norm_freq'],
    Norm_freq1=ref_123_df['Norm_freq1']-ref_123_df['Norm_freq'],
    Norm_freq2=ref_123_df['Norm_freq2']-ref_123_df['Norm_freq'],
    Norm_freq3=ref_123_df['Norm_freq3']-ref_123_df['Norm_freq'],
    Translation=ref_123_df['Translation'],
))

hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("Translation", "@Translation"),
    ("Norm_freq1", "@Norm_freq1"),
    ("Norm_freq2", "@Norm_freq2"),
    ("Norm_freq3", "@Norm_freq3")
])

# Create a linear color map that maps the frequency values to a red 8 point gradient, 
# but only use the top 7 in reverse so that dark red is highest and the lowest is just off white
color_mapper = linear_cmap(field_name='Norm_freq1', palette=Reds[8][6::-1], low=0, high=1)

# Create a scatter plot of the embeddings with the points colored by frequency
plot = figure(tools=['pan,wheel_zoom,box_zoom,reset',hover],title='UMAP Embeddings of Protein Sequences')
plot.scatter(x='x', y='y', source=source, color=color_mapper)
plot.add_tools(LassoSelectTool())

# Add a color bar to show the mapping between frequency and color
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0, 0))
plot.add_layout(color_bar, 'right')

# create a table to display the selected points
columns = [TableColumn(field=col, title=col) for col in ref_123_df.columns]
table = DataTable(source=source, columns=columns, width=400, height=400)


# combine the plot, tables, and button into a layout
layout = column(row(plot, table))

# display the layout
show(layout)


### Nucleotide-based analysis
There appear to be satellite clusters around a major point when looking at individual datasets, which suggests to me that there were a lot of mutations during library prep, evolution, sequencing, etc.  Even points just one amino acid away from the major point is almost negligable when it comes to frequency.  This effect may be real and changing one amino acid from the unique sequence could stop functional activity; but it's likely an artifact of sequencing, as not every position is likely to be critical to function.  Therefore, I'll attempt to use a distance measure to assign each variant to it's closest reference sequence at the nucleotide level. 


| Model | Description | Substitutions | Indels |
| --- | --- | --- | --- |
| Hamming distance | measures the number of substitutions from one string to another. Requires fixed length| X ||
|Levenshtein distance | measures edit distance between strings (Best for this use case because we expect artifact mutations) | X | X | 
|Smith-Waterman algorithm| identifies regions of similarity between two sequences | X | X |
|Needleman-Wunsch algorithm| global alignment to compare two sequences and identify regions of similarity| X | X | X |
|Blosum Matrix| Substitution matrix that assigns scores to each possible amino acid substitution based on the frequency of occurrence of the substitution in known protein sequences | X|X|
|Phylogenetic analysis| Compares multiple amino acid sequences via construction of a phylogenetic tree and measures branch lengths ||


In [37]:
#this takes a while, so I had done it one at a time
from Levenshtein import distance

# Function to calculate edit distance between two sequences
def calculate_edit_distance(seq1, seq2):
    return distance(seq1, seq2)

# Assign each subject sequence to the closest reference sequence
lib1['closest_reference1'] = lib1['Genotype'].apply(lambda x: min(reference['Genotype'], key=lambda y: calculate_edit_distance(x, y)))
#lib2['closest_reference2'] = lib2_filtered['Genotype'].apply(lambda x: min(reference['Genotype'], key=lambda y: calculate_edit_distance(x, y)))
#lib3['closest_reference3'] = lib3['Genotype'].apply(lambda x: min(reference['Genotype'], key=lambda y: calculate_edit_distance(x, y)))




Lib1 assigned nucleotide count

In [39]:
#calculate relative proportion of membership
lib1_recount=lib1['closest_reference1'].value_counts(normalize=True).to_frame()

freq_min, freq_max = min(lib1_recount['closest_reference1']), max(lib1_recount['closest_reference1'])
lib1_recount['Norm_freq'] = lib1_recount['closest_reference1'].apply(normalize_freq)
#label the library
lib1_recount['Library']=1
lib1_recount.reset_index(inplace=True)
lib1_recount=lib1_recount.drop('closest_reference1', axis=1)
lib1_recount.columns=['Genotype', 'Norm_freq1', 'Library']

lib1_recount

,Genotype,Norm_freq1,Library
0,GGCAACAACAACAATATATATTCCCTAGGCAGTACGGTTGGGACTAATCATGCACCTTAA,1.0000,1
1,CGCTATCTCCTGCGCGACTATCGCGGCGGCGGACACAAGTCCATGCTGCGCAATCCATGA,0.8389,1
2,ATAGTGAGCACATCGCTGTATTACCGCAGGCGTGGCGCTGCACCGTTACTCCAGCCGTAA,0.8079,1
3,TGTATCGTCATCAACTGGTGGTTCTATCTACGCCCAGGTGCTGAGCATCATAACCCTTAA,0.3649,1
4,GCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACGCTTCCCTAA,0.2025,1
...,...,...,...
1470,CATGCCTATGAGAATCACTACGGCAAAATTACCAGTGATCCTTCAGGCGACTATCCGTAA,0.0000,1
1471,GTAGAAACTCCTGCTAAATTTGAAGTGGTTCCCCATGCCTTGAATGTCATTATCCCCTAA,0.0000,1
1472,GTAAAAGAGGGAGATGTGTATCGAAAGACGACTCGGAGCTGGGGAGCGGGCATCCCATAG,0.0000,1
1473,CATTCGCTAAGGTGGCGATTCAAAAATCAGTCAAAAAAATCTTCAACCGATACACCGTAA,0.0000,1


Lib2 assigned nucleotide count

In [40]:
#calculate relative proportion of membership
lib2_recount=lib2_filtered['closest_reference2'].value_counts(normalize=True).to_frame()

freq_min, freq_max = min(lib2_recount['closest_reference2']), max(lib2_recount['closest_reference2'])
lib2_recount['Norm_freq'] = lib2_recount['closest_reference2'].apply(normalize_freq)
#label the library
lib2_recount['Library']=2
lib2_recount.reset_index(inplace=True)
lib2_recount=lib2_recount.drop('closest_reference2', axis=1)
lib2_recount.columns=['Genotype', 'Norm_freq2', 'Library']



lib2_recount

,Genotype,Norm_freq2,Library
0,GGCAACAACAACAATATATATTCCCTAGGCAGTACGGTTGGGACTAATCATGCACCTTAA,1.0000,2
1,CGCTATCTCCTGCGCGACTATCGCGGCGGCGGACACAAGTCCATGCTGCGCAATCCATGA,0.7843,2
2,ACAGGTGGTGTTTTGTTTTGGGCGTTCAAAATAGGATTACTAAAAAAGAAACCCCCGTGA,0.6176,2
3,CGCAATTCGGATGGTTTTTTCCACAATTTGATTGACTCATTTAAAAAGAAAAAGCCATGA,0.5882,2
4,ATAGTGAGCACATCGCTGTATTACCGCAGGCGTGGCGCTGCACCGTTACTCCAGCCGTAA,0.4314,2
5,GCGGGATTGCAGGCTGCGTGGGGCGTCTATCAGGCCGAGAATGGCGTGACGCTTCCCTAA,0.3529,2
6,AAATCCCCACATGGAGTCTATTGTACAACAAACCTTAGCAACAATCGAATGATTCCATGA,0.2451,2
7,ATGAACACCCTCGGCTGGTTTGATTTTAAATCGGCGTGGTTTGCGCCGCCGGAACCCTAG,0.2353,2
8,TGTATCGTCATCAACTGGTGGTTCTATCTACGCCCAGGTGCTGAGCATCATAACCCTTAA,0.2059,2
9,AAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,0.1765,2


Lib3 assigned nucleotide count

In [41]:
#calculate relative proportion of membership
lib3_recount=lib3['closest_reference3'].value_counts(normalize=True).to_frame()

freq_min, freq_max = min(lib3_recount['closest_reference3']), max(lib3_recount['closest_reference3'])
lib3_recount['Norm_freq'] = lib3_recount['closest_reference3'].apply(normalize_freq)
#label the library
lib3_recount['Library']=1
lib3_recount.reset_index(inplace=True)
lib3_recount=lib3_recount.drop('closest_reference3', axis=1)
lib3_recount.columns=['Genotype', 'Norm_freq3', 'Library']

lib3_recount

,Genotype,Norm_freq3,Library
0,CGCTATCTCCTGCGCGACTATCGCGGCGGCGGACACAAGTCCATGCTGCGCAATCCATGA,1.0000,1
1,AAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,0.5319,1
2,GGCAACAACAACAATATATATTCCCTAGGCAGTACGGTTGGGACTAATCATGCACCTTAA,0.4347,1
3,ATAGTGAGCACATCGCTGTATTACCGCAGGCGTGGCGCTGCACCGTTACTCCAGCCGTAA,0.4179,1
4,CGCAATTCGGATGGTTTTTTCCACAATTTGATTGACTCATTTAAAAAGAAAAAGCCATGA,0.2112,1
...,...,...,...
499,GAGTCGGCCATCGACCTGTTCCTGGCCGGCTACGAAGTCAGCCAATCCCGCCAACCGTAA,0.0000,1
500,GGAAGCCTTGCTAAGATTTATCGTAAGCATGGATTGACACCTGCCACAAGAAAACCCTAA,0.0000,1
501,GCCGCACAGGTCAGGCAATGGCTTGCGCTTCTTTCAACGCCAGCTGCAGCTTCCCCATAA,0.0000,1
502,GTAGGTGTTTCCACAATTTATAGAAAATTTCCGGCGAATGAGAGCAATGAATCGCCCTGA,0.0000,1


Combine the dataframes

In [42]:
# inner join
ref_1_redf = pd.merge(reference, lib1_recount, on='Genotype', how='left')
ref_12_redf = pd.merge(ref_1_redf, lib2_recount, on='Genotype', how='left')
ref_123_redf = pd.merge(ref_12_redf, lib3_recount, on='Genotype', how='left').fillna(0)

# display dataframe
ref_123_redf.sort_values('Norm_freq3')

,Genotype,reads,Translation,Frequency,Norm_freq,Norm_freq1,Library_x,Norm_freq2,Library_y,Norm_freq3,Library
1623,TTCACTCTCAAAGACCATTATCAAAACCTAGCTAAGATTCTCTCTGTGGGACACCCTTAG,323,FTLKDHYQNLAKILSVGHP,0.000160,0.1032,0.0000,0.0,0.0000,0.0,0.0000,0.0
2141,GTAAAAGAAAAGACAAGCTGGATTAACATTCCAGTTTGTCTTCAATCTGAGATACCATAG,292,VKEKTSWINIPVCLQSEIP,0.000145,0.0933,0.0000,0.0,0.0000,0.0,0.0000,0.0
2142,CACGCCAATTTACTGCGCTTCTTGTCGGAACTACTTGCTCAGGTCTTTGGGAGGCCGTAA,999,HANLLRFLSELLAQVFGRP,0.000496,0.3191,0.0000,0.0,0.0000,0.0,0.0000,0.0
2143,GCGGAGCAGAGCGCGCCCTATGGGACGCCGACATTGTTGGCACGCGCCGACGTGCCCTGA,1493,AEQSAPYGTPTLLARADVP,0.000741,0.4768,0.0001,1.0,0.0000,0.0,0.0000,0.0
2144,GAAGACAACCCGAGCGGCTTTCTGCTTGGTCGGGACAAGTTGCAGGAGTTCACCCCATGA,648,EDNPSGFLLGRDKLQEFTP,0.000322,0.2070,0.0000,0.0,0.0000,0.0,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
30,CGCAATTCGGATGGTTTTTTCCACAATTTGATTGACTCATTTAAAAAGAAAAAGCCATGA,408,RNSDGFFHNLIDSFKKKKP,0.000202,0.1303,0.0030,1.0,0.5882,2.0,0.2112,1.0
1592,ATAGTGAGCACATCGCTGTATTACCGCAGGCGTGGCGCTGCACCGTTACTCCAGCCGTAA,1138,IVSTSLYYRRRGAAPLLQP,0.000565,0.3635,0.8079,1.0,0.4314,2.0,0.4179,1.0
829,GGCAACAACAACAATATATATTCCCTAGGCAGTACGGTTGGGACTAATCATGCACCTTAA,359,GNNNNIYSLGSTVGTNHAP,0.000178,0.1147,1.0000,1.0,1.0000,2.0,0.4347,1.0
2773,AAGGTACTCCGACCCAAATTTGGAACCCTGCTCTCCCTCGCCTATTTCCTCCTTCCTTAG,849,KVLRPKFGTLLSLAYFLLP,0.000421,0.2712,0.1299,1.0,0.1765,2.0,0.5319,1.0


Run through the OHE and mapping with the new sequences, plot with translation mouseover

In [43]:
sequences = list(reference['Genotype'])
sequences


['...',
 ...]

In [44]:
nucs = 'ATCG-'
num_nucs = len(nucs)


max_len = 63  # set the maximum length of the sequences
pad_char = "-"  # choose the padding character
pad_pos="left"

def one_hot_encode(seq):
    seq_len = len(seq)
    if seq_len > max_len:
        raise ValueError(f"Sequence {seq} is longer than the maximum length {max_len}")
    pad_len = max_len - seq_len
    if pad_len > 0:
        if pad_pos == "left":
            seq = pad_char * pad_len + seq
        else:
            seq = seq + pad_char * pad_len
    encoding = np.zeros((max_len, num_nucs))
    for i, acid in enumerate(seq):
        if acid not in nucs:
            raise ValueError(f"Invalid amino acid {acid} in sequence {seq}")
        encoding[i, nucs.index(acid)] = 1
    return encoding
    
one_hot_sequences = [one_hot_encode(seq) for seq in sequences]

In [45]:
flattened_sequences = [seq.flatten() for seq in one_hot_sequences]

In [46]:
umap_object = umap.UMAP()
umap_embeddings = umap_object.fit_transform(flattened_sequences)

In [48]:
umap_df_pep= pd.DataFrame(data=dict(
    x=umap_embeddings[:, 0],
    y=umap_embeddings[:, 1],
    Matched_Genotype=ref_123_redf['Genotype'],
    Norm_freq=ref_123_redf['Norm_freq'],
    Norm_freq1=ref_123_redf['Norm_freq1']-ref_123_redf['Norm_freq'],
    Norm_freq2=ref_123_redf['Norm_freq2']-ref_123_redf['Norm_freq'],
    Norm_freq3=ref_123_redf['Norm_freq3']-ref_123_redf['Norm_freq'],
    Translation=ref_123_redf['Translation']
    ))

umap_df_pep.to_csv('./output/umap_df_nuc.csv')

In [49]:
# Create a Bokeh ColumnDataSource with the embeddings and frequency values
source = ColumnDataSource(data=dict(
    x=umap_embeddings[:, 0],
    y=umap_embeddings[:, 1],
    Norm_freq=ref_123_redf['Norm_freq'],
    Norm_freq1=ref_123_redf['Norm_freq1']-ref_123_redf['Norm_freq'],
    Norm_freq2=ref_123_redf['Norm_freq2']-ref_123_redf['Norm_freq'],
    Norm_freq3=ref_123_redf['Norm_freq3']-ref_123_redf['Norm_freq'],
    Translation=ref_123_redf['Translation']
))

hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("Translation", "@Translation"),
    ("Norm_freq1", "@Norm_freq1"),
    ("Norm_freq2", "@Norm_freq2"),
    ("Norm_freq3", "@Norm_freq3")
])

# Create a linear color map that maps the frequency values to a red 8 point gradient, 
# but only use the top 7 in reverse so that dark red is highest and the lowest is just off white
color_mapper = linear_cmap(field_name='Norm_freq1', palette=Reds[8][6::-1], low=0, high=1)

# Create a scatter plot of the embeddings with the points colored by frequency
plot = figure(tools=['pan,wheel_zoom,box_zoom,reset',hover],title='UMAP Embeddings of Protein Sequences')
plot.scatter(x='x', y='y', source=source, color=color_mapper)
plot.add_tools(LassoSelectTool())

# Add a color bar to show the mapping between frequency and color
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0, 0))
plot.add_layout(color_bar, 'right')

# create a table to display the selected points
columns = [TableColumn(field=col, title=col) for col in ref_123_redf.columns]
table = DataTable(source=source, columns=columns, width=400, height=400)

# create a button to save the selected points as a CSV file
button = Button(label="Save CSV", button_type="success")

# define a callback function for the button click
def save_csv():
    selected_indices = source.selected.indices
    if selected_indices:
        selected_data = {col: source.data[col][selected_indices] for col in source.column_names}
        selected_df = pd.DataFrame(selected_data)
        selected_df.to_csv("selected_points.csv", index=False)

button.on_click(save_csv)

# define a callback function for the selection change event
def selection_change_callback(attrname, old, new):
    selected_indices = source.selected.indices
    if selected_indices:
        selected_data = {col: source.data[col][selected_indices] for col in source.column_names}
        selected_source.data = selected_data
    else:
        selected_source.data = {col: [] for col in source.column_names}

# create a new data source for the selected points
selected_source = ColumnDataSource({col: [] for col in source.column_names})
selected_table = DataTable(source=selected_source, columns=columns, width=400, height=400)

# add a selection change callback to the data source
source.selected.on_change('indices', selection_change_callback)


# combine the plot, tables, and button into a layout
layout = column(row(plot, table, button))

# display the layout
show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



Now, standalone apps can be run for the peptide sequences or the clustered nucleotide sequences. Variants of interest can be selected using the lasso tool and extracted to a CSV.

1. Using the terminal, activate the virtual environment with bokeh 
2. Set the library to the one you want to view 
3. Enter the command from the main folder: 
    - bokeh serve --show ./code/nuc_app.py 
    - bokeh serve --show ./code/pep_app.py